In [1]:
# Imports

import requests
from bs4 import BeautifulSoup
import re
from playwright.async_api import async_playwright
import pandas as pd
import time
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

In [2]:
df = pd.read_csv('../data/judiciary.csv')

df.head()

,name,position,docs,doc_links,date,orgs,district
0,Gordon P. Gallagher,united states district judge for the district of colorado,"['Colorado DA Rubinstein Support For Gallagher', 'Former MCBA Presidents And Attorneys Support For Gallagher', 'Former CO US Attorneys Support For Gallagher', 'Grand Junction Legal Community Support For Gallagher', 'HAP And Mesa County Libraries Support For Gallagher', 'Mesa County DAs And Sheriffs Support For Gallagher', 'Riverside Educational Center Support For Gallagher', 'Ute Mountain Ute Tribe Support For Gallagher', 'Western CO Attorneys Support For Gallagher']","['https://www.judiciary.senate.gov/colorado-da-rubinstein-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/former-mcba-presidents-and-attorneys-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/former-co-us-attorneys-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/grand-junction-legal-community-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/hap-and-mesa-county-libraries-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/mesa-county-das-and-sheriffs-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/riverside-educational-center-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/ute-mountain-ute-tribe-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/western-co-attorneys-support-for-gallagher&download=1']","Tuesday, December 13, 2022","['Colorado DA Rubinstein', 'Former MCBA Presidents And Attorneys', 'Former CO US Attorneys', 'Grand Junction Legal Community', 'HAP And Mesa County Libraries', 'Mesa County DAs And Sheriffs', 'Riverside Educational Center', 'Ute Mountain Ute Tribe', 'Western CO Attorneys']",district of colorado
1,P. Casey Pitts,united states district judge for the northern district of california,"['Former AAG Eric Dreiband Support For Pitts', 'Stephen Cannon Support For Pitts', 'National Education Association Support For Pitts', 'William Baude Support For Pitts', 'FCC Deputy GC Boizelle Support For Pitts']","['https://www.judiciary.senate.gov/former-aag-eric-dreiband-support-for-pitts&download=1', 'https://www.judiciary.senate.gov/stephen-cannon-support-for-pitts&download=1', 'https://www.judiciary.senate.gov/national-education-association-support-for-pitts&download=1', 'https://www.judiciary.senate.gov/william-baude-support-for-pitts&download=1', 'https://www.judiciary.senate.gov/fcc-deputy-gc-boizelle-support-for-pitts&download=1']","Tuesday, December 13, 2022","['Former AAG Eric Dreiband', 'Stephen Cannon', 'National Education Association', 'William Baude', 'FCC Deputy GC Boizelle']",northern district of california
2,Andrew G. Schopler,united states district judge for the southern district of california,"['Major General Mark Malanka Support For Schopler', 'Judge Irma Gonzalez Support For Schopler', 'Former Law Clerks Letter Of Support For Schopler', 'Harvard Law School Classmates Letter Of Support For Schopler']","['https://www.judiciary.senate.gov/major-general-mark-malanka-support-for-schopler&download=1', 'https://www.judiciary.senate.gov/judge-irma-gonzalez-support-for-schopler&download=1', 'https://www.judiciary.senate.gov/former-law-clerks-letter-of-support-for-schopler&download=1', 'https://www.judiciary.senate.gov/harvard-law-school-classmates-letter-of-support-for-schopler&download=1']","Tuesday, December 13, 2022","['Major General Mark Malanka', 'Judge Irma Gonzalez', 'Former Law Clerks', 'Harvard Law School Classmates']",southern district of california
3,Arun Subramanian,united states district judge for the southern district of new york,"['Former SCOTUS Clerks Support For Subramanian', 'SABANY/AABANY/SABANA Support For Subramanian', 'Colleagues At Susman Godfrey Support For Subramanian', 'Columbia Law Review Board Support For Subramanian', 'Co-Counsel Eric Havian Support For Subramanian', 'Columbia Law Review Support For Subramanian']","['https://www.judic

In [3]:
# So! At this point I want to get the elements of the orgs list into their own frame. And saving the dataframe as a file has turned the lists into strings, so I need to turn them back into lists.

for i in range(len(df)):
    df['orgs'][i] = df['orgs'][i].split(',')
    df['orgs'][i] = [x.strip() for x in df['orgs'][i]]

for i in range(len(df)):
    df['orgs'][i] = [x.replace('[','') for x in df['orgs'][i]]
    df['orgs'][i] = [x.replace(']','') for x in df['orgs'][i]]

In [4]:
for i in range(len(df)):
    df['docs'][i] = df['docs'][i].split(',')
    df['docs'][i] = [x.strip() for x in df['docs'][i]]

for i in range(len(df)):
    df['docs'][i] = [x.replace('[','') for x in df['docs'][i]]
    df['docs'][i] = [x.replace(']','') for x in df['docs'][i]]

In [5]:
for i in range(len(df)):
    df['doc_links'][i] = df['doc_links'][i].split(',')
    df['doc_links'][i] = [x.strip() for x in df['doc_links'][i]]

for i in range(len(df)):
    df['doc_links'][i] = [x.replace('[','') for x in df['doc_links'][i]]
    df['doc_links'][i] = [x.replace(']','') for x in df['doc_links'][i]]

In [6]:
df.head()

,name,position,docs,doc_links,date,orgs,district
0,Gordon P. Gallagher,united states district judge for the district of colorado,"['Colorado DA Rubinstein Support For Gallagher', 'Former MCBA Presidents And Attorneys Support For Gallagher', 'Former CO US Attorneys Support For Gallagher', 'Grand Junction Legal Community Support For Gallagher', 'HAP And Mesa County Libraries Support For Gallagher', 'Mesa County DAs And Sheriffs Support For Gallagher', 'Riverside Educational Center Support For Gallagher', 'Ute Mountain Ute Tribe Support For Gallagher', 'Western CO Attorneys Support For Gallagher']","['https://www.judiciary.senate.gov/colorado-da-rubinstein-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/former-mcba-presidents-and-attorneys-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/former-co-us-attorneys-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/grand-junction-legal-community-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/hap-and-mesa-county-libraries-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/mesa-county-das-and-sheriffs-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/riverside-educational-center-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/ute-mountain-ute-tribe-support-for-gallagher&download=1', 'https://www.judiciary.senate.gov/western-co-attorneys-support-for-gallagher&download=1']","Tuesday, December 13, 2022","['Colorado DA Rubinstein', 'Former MCBA Presidents And Attorneys', 'Former CO US Attorneys', 'Grand Junction Legal Community', 'HAP And Mesa County Libraries', 'Mesa County DAs And Sheriffs', 'Riverside Educational Center', 'Ute Mountain Ute Tribe', 'Western CO Attorneys']",district of colorado
1,P. Casey Pitts,united states district judge for the northern district of california,"['Former AAG Eric Dreiband Support For Pitts', 'Stephen Cannon Support For Pitts', 'National Education Association Support For Pitts', 'William Baude Support For Pitts', 'FCC Deputy GC Boizelle Support For Pitts']","['https://www.judiciary.senate.gov/former-aag-eric-dreiband-support-for-pitts&download=1', 'https://www.judiciary.senate.gov/stephen-cannon-support-for-pitts&download=1', 'https://www.judiciary.senate.gov/national-education-association-support-for-pitts&download=1', 'https://www.judiciary.senate.gov/william-baude-support-for-pitts&download=1', 'https://www.judiciary.senate.gov/fcc-deputy-gc-boizelle-support-for-pitts&download=1']","Tuesday, December 13, 2022","['Former AAG Eric Dreiband', 'Stephen Cannon', 'National Education Association', 'William Baude', 'FCC Deputy GC Boizelle']",northern district of california
2,Andrew G. Schopler,united states district judge for the southern district of california,"['Major General Mark Malanka Support For Schopler', 'Judge Irma Gonzalez Support For Schopler', 'Former Law Clerks Letter Of Support For Schopler', 'Harvard Law School Classmates Letter Of Support For Schopler']","['https://www.judiciary.senate.gov/major-general-mark-malanka-support-for-schopler&download=1', 'https://www.judiciary.senate.gov/judge-irma-gonzalez-support-for-schopler&download=1', 'https://www.judiciary.senate.gov/former-law-clerks-letter-of-support-for-schopler&download=1', 'https://www.judiciary.senate.gov/harvard-law-school-classmates-letter-of-support-for-schopler&download=1']","Tuesday, December 13, 2022","['Major General Mark Malanka', 'Judge Irma Gonzalez', 'Former Law Clerks', 'Harvard Law School Classmates']",southern district of california
3,Arun Subramanian,united states district judge for the southern district of new york,"['Former SCOTUS Clerks Support For Subramanian', 'SABANY/AABANY/SABANA Support For Subramanian', 'Colleagues At Susman Godfrey Support For Subramanian', 'Columbia Law Review Board Support For Subramanian', 'Co-Counsel Eric Havian Support For Subramanian', 'Columbia Law Review Support For Subramanian']","['https://www.judic

In [7]:
# Almost forgot.

pd.to_datetime(df['date'])

0    2022-12-13
1    2022-12-13
2    2022-12-13
3    2022-12-13
4    2022-11-30
5    2022-11-30
6    2022-11-30
7    2022-11-15
8    2022-11-15
9    2022-11-15
10   2022-11-15
11   2022-11-15
12   2022-10-12
13   2022-10-12
14   2022-10-12
15   2022-10-12
16   2022-10-12
17   2022-09-21
18   2022-09-21
19   2022-09-21
20   2022-09-07
21   2022-09-07
22   2022-09-07
23   2022-09-07
24   2022-07-27
25   2022-07-27
26   2022-07-27
27   2022-06-22
28   2022-06-22
29   2022-05-25
30   2022-05-25
31   2022-05-11
32   2022-05-11
33   2022-05-11
34   2022-04-27
35   2022-04-27
36   2022-03-02
37   2022-03-02
38   2022-03-02
39   2022-02-16
40   2022-02-16
41   2022-02-16
42   2022-02-01
43   2022-02-01
44   2022-02-01
45   2022-01-12
46   2022-01-12
47   2022-01-12
48   2021-12-15
49   2021-12-15
50   2021-12-15
51   2021-12-15
52   2021-12-15
53   2021-12-01
54   2021-12-01
55   2021-12-01
56   2021-12-01
57   2021-11-17
58   2021-11-17
59   2021-11-17
60   2021-11-03
61   2021-11-03
62   202

In [8]:
# Lovely. Now I want to get the orgs into their own dataframe.

org_list = []

for i in range(len(df)):
    for org in df['orgs'][i]:
       # if org != '':
        temp_dict = {
            'org': org.replace("'",''),
            'name': df['name'][i],
            'position': df['position'][i],
            'date': df['date'][i],
            'district': df['district'][i]
        }
        temp_df = pd.DataFrame(temp_dict, index=[0])
        org_list.append(temp_df)

In [9]:
org_df = pd.concat(org_list, ignore_index=True)
org_df.head()

,org,name,position,date,district
0,Colorado DA Rubinstein,Gordon P. Gallagher,united states district judge for the district of colorado,"Tuesday, December 13, 2022",district of colorado
1,Former MCBA Presidents And Attorneys,Gordon P. Gallagher,united states district judge for the district of colorado,"Tuesday, December 13, 2022",district of colorado
2,Former CO US Attorneys,Gordon P. Gallagher,united states district judge for the district of colorado,"Tuesday, December 13, 2022",district of colorado
3,Grand Junction Legal Community,Gordon P. Gallagher,united states district judge for the district of colorado,"Tuesday, December 13, 2022",district of colorado
4,HAP And Mesa County Libraries,Gordon P. Gallagher,united states district judge for the district of colorado,"Tuesday, December 13, 2022",district of colorado


In [10]:
org_df.org.value_counts()

                                                                     17
AFJ                                                                  12
NAPABA                                                                8
PFAW                                                                  7
NOBLE                                                                 5
NJSBA                                                                 4
Former Law Clerks                                                     4
NAPAWF                                                                3
Former Federal Prosecutors                                            3
Former Prosecutors                                                    3
Opposing Counsel                                                      3
LCCHR                                                                 3
Former Opposing Counsel                                               2
Ouzinkie Tribal Council                                         

In [11]:
# Would like to remove blanks, if they aren't mistakes.

names = org_df[org_df.org == ''].name
df[df.name.isin(names)]

,name,position,docs,doc_links,date,orgs,district
24,Frances Kay Behm,united states district judge for the eastern district of michigan,['Responses To Questions For The Record - Behm - 2022-07-27'],['https://www.judiciary.senate.gov/responses-to-questions-for-the-record---behm---2022-07-27&download=1'],"Wednesday, July 27, 2022",[],eastern district of michigan
31,Stephen Henley Locher,united states district judge for the southern district of iowa,['Responses To Questions For The Record - Locher - 2022-05-11'],['https://www.judiciary.senate.gov/responses-to-questions-for-the-record---locher---2022-05-11&download=1'],"Wednesday, May 11, 2022",[],southern district of iowa
36,Robert Steven Huie,united states district judge for the southern district of california,['Robert Huie Written Responses To Questions For The Record'],['https://www.judiciary.senate.gov/robert-huie-written-responses-to-questions-for-the-record&download=1'],"Wednesday, March 2, 2022",[],southern district of california
45,Jessica G.L. Clarke,united states district judge for the southern district of new york,['Clarke Responses To Written Questions For The Record'],['https://www.judiciary.senate.gov/clarke-responses-to-written-questions-for-the-record&download=1'],"Wednesday, January 12, 2022",[],southern district of new york
46,Hector Gonzalez,united states district judge for the eastern district of new york,['Gonzalez Responses To Written Questions For The Record'],['https://www.judiciary.senate.gov/gonzalez-responses-to-written-questions-for-the-record&download=1'],"Wednesday, January 12, 2022",[],eastern district of new york
47,Fred W. Slaughter,united states district judge for the central district of california,['Slaughter Responses To Written Questions For The Record'],['https://www.judiciary.senate.gov/slaughter-responses-to-written-questions-for-the-record&download=1'],"Wednesday, January 12, 2022",[],central district of california
48,Georgette Castner,united states district judge for the district of new jersey,['Georgette Castner Responses To Questions For The Record'],['https://www.judiciary.senate.gov/georgette-castner-responses-to-questions-for-the-record&download=1'],"Wednesday, December 15, 2021",[],district of new jersey
50,Julie Rebecca Rubin,united states district judge for the district of maryland,['Julie Rubin Responses To Questions For The Record'],['https://www.judiciary.senate.gov/julie-rubin-responses-to-questions-for-the-record&download=1'],"Wednesday, December 15, 2021",[],district of maryland
54,Jacqueline Scott Corley,united states district judge for the northern district of california,['Corley Response S To Questions For The Record'],['https://www.judiciary.senate.gov/corley-response-s-to-questions-for-the-record&download=1'],"Wednesday, December 1, 2021",[],northern district of california
57,Bridget Meehan Brennan,united states district judge for the western district of washington,['Bridget Brennan Responses To Questions For The Record'],['https://www.judiciary.senate.gov/bridget-brennan-responses-to-questions-for-the-record&download=1'],"Wednesday, November 17, 2021",[],western district of washington


In [12]:
# These are just Q&A docs, so I'm going to drop them

org_df = org_df[org_df.org != '']

In [13]:
org_df.org.value_counts()

AFJ                                                                  12
NAPABA                                                                8
PFAW                                                                  7
NOBLE                                                                 5
NJSBA                                                                 4
Former Law Clerks                                                     4
Former Federal Prosecutors                                            3
LCCHR                                                                 3
Former Prosecutors                                                    3
Opposing Counsel                                                      3
NAPAWF                                                                3
Washington State Gambling Commission                                  2
Suquamish Tribe                                                       2
Spokane Tribe                                                   

In [14]:
# Lovely. 

org_df.to_csv('../output/orgs.csv', index=False)